In [2]:
from dask_kubernetes import KubeCluster

In [ ]:
# Initial attempt at creation (also failed do to networking)
#tag::create_in_default[]
cluster = KubeCluster.from_yaml('worker-spec.yaml')
#end::create_in_default[]

In [5]:
# This is the one where it failed because I was running it outside a cluster and it could not communicate
#tag::create_in_namespace[]
cluster = KubeCluster.from_yaml('worker-spec.yaml', namespace='dask')
#end::create_in_namespace[]

/home/holden/.local/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33015 instead
  warnings.warn(
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://192.168.1.38:39555
distributed.scheduler - INFO -   dashboard at:                    :33015


In [3]:
cluster.adapt(minimum=1, maximum=100)

distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=100


In [5]:
# Example usage
from dask.distributed import Client
import dask.array as da

# Connect Dask to the cluster
client = Client(cluster)


distributed.scheduler - INFO - Receive client connection: Client-64b67454-0776-11eb-a73d-b88584aac7ce
distributed.core - INFO - Starting established connection


In [6]:
client.scheduler_comm.comm.handshake_info()

{'compression': 'lz4', 'python': (3, 8, 0), 'pickle-protocol': 5}

In [7]:
# Create a large array and calculate the mean
array = da.ones((1000, 1000, 1000))
print(array.mean().compute())  # Should print 1.0|

distributed.scheduler - ERROR - Couldn't gather keys {"('mean_agg-aggregate-bd6b5d0ba8d5dce415f74feccf68818e',)": ['tcp://10.42.2.10:43251']} state: ['memory'] workers: ['tcp://10.42.2.10:43251']
NoneType: None
distributed.scheduler - INFO - Remove worker <Worker 'tcp://10.42.2.10:43251', name: 0, memory: 1, processing: 0>
distributed.core - INFO - Removing comms to tcp://10.42.2.10:43251
distributed.scheduler - INFO - Lost all workers
distributed.scheduler - ERROR - Workers don't have promised key: ['tcp://10.42.2.10:43251'], ('mean_agg-aggregate-bd6b5d0ba8d5dce415f74feccf68818e',)
NoneType: None
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('mean_agg-aggregate-bd6b5d0ba8d5dce415f74feccf68818e',)": ('tcp://10.42.2.10:43251',)}
distributed.scheduler - INFO - Register worker <Worker 'tcp://10.42.2.10:43251', name: 0, memory: 1, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.42.2.10:43251
distributed.core - INFO - Starting 

KeyboardInterrupt: 